#   EDSA: MOVIE RECOMMENDATION 2022  
*unsupervised learning*

© Explore Data Science Academy

#  2201FT_Team GM2  

Honour Code
We {***Team GM2***}, confirm - by submitting this document - that the solutions in this notebook are a result of our own work and that we abide by the EDSA honour code.

Non-compliance with the honour code constitutes a material breach of contract.

#  MEET THE TEAM!  
[Lindiwe Songelwa](https://www.linkedin.com/in/lindiwe-songelwa/)

[Thabang Pardon Mokoena](https://www.linkedin.com/in/thabang-pardon-mokoena/)

[Sizakele Beauty Mtsweni](https://www.linkedin.com/in/sizakele-beuty-mtsweni-236935172/)

[Nkoka Khosa](https://www.linkedin.com/)

[Elizabeth Matlala](https://www.linkedin.com/)

[Kanego Kgabalo](https://www.linkedin.com/)


# PREDICT OVERVIEW

*Recommender systems are the systems that are designed to recommend things to the user based on many different factors, they deal with a large volume of information present by filtering the most important information based on the data provided by a user and other factors that take care of the user’s preference and interest.*
 
 *Having the ability to predict user ratings, even before the user has provided one, makes recommender systems a powerful tool. The recommender system analyzes and finds items with similar user engagement data by filtering. It uses different analysis methods such as batch analysis, real-time analysis, or near-real-time system analysis.*
 
 *Recommender systems can be a powerful tool for any e-commerce business, and rapid future developments in the field will increase their business value even further.*


#  PROBLEM STATEMENT 

> Create a collaborative or content based filtering algorithm that will predict how a user will rate a movie they have not yet seen
 

![](https://wallpaperaccess.com/full/4839516.jpg)

<a id="cont"></a>

## <u> Table of Contents </u> 

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Important Functions</a>

<a href=#three>3. Loading and Pre-processing Data</a>

<a href=#four>4. Exploratory Data Analysis (EDA)</a>

<a href=#five>5. Model Explanations</a>

<a href=#six>6. Modelling</a>

<a href=#seven>7. Model Performance / Validation</a>

<a href=#eight>8. Reference</a>

 <a id="one"></a>
## 1. Importing Packages
<a href=#cont>Back to Table of Contents</a>

---
    
| 📽 Description: Importing Packages 📽 |
| :--------------------------- |
| In this section we will import, and briefly discuss, the libraries that will be used throughout our analysis and modelling. |

---

In [ ]:
# Import our regular old heroes 
import numpy as np
import pandas as pd
import scipy as sp # <-- The sister of Numpy, used in our code for numerical efficientcy. 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import iplot 
from PIL import Image
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import re

# Entity featurization and similarity computation
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.feature_extraction.text import TfidfVectorizer

#NLP libraries
from nltk.corpus import stopwords
import string
import nltk

# Models
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, NMF, SlopeOne, CoClustering,KNNWithMeans,KNNBasic,accuracy
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from surprise.model_selection import cross_validate,train_test_split
 
# Performance Evaluation
from surprise import accuracy
from sklearn.metrics import mean_squared_error
from surprise.model_selection import GridSearchCV, cross_validate, train_test_split

# Display
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")
pd.set_option('display.max_columns', 37)

In [ ]:
# This package makes the algorithm run faster
!pip install git+https://github.com/Explore-Unsupervised-Learning-TEAM-GM2/Surprise

<div class="alert alert-block alert-success"> 
 <b>Data description:</b> 

This dataset consists of several million 5-star ratings obtained from users of the online MovieLens movie recommendation service. The MovieLens dataset has long been used by industry and academic researchers to improve the performance of explicitly-based recommender systems, and now you get to as well!

For this Predict, we'll be using a special version of the MovieLens dataset which has enriched with additional data, and resampled for fair evaluation purposes.

Our datasets is divided into numeric and categorical data, where;
* Categorical data is defined as a type of data that is used to group information with similar characteristics while
* Numerical data is defined a type of data that expresses information in the form of numbers.
</div>     

 <a id="two"></a>
## 2. Important Functions
<a href=#cont>Back to Table of Contents</a>

---
    
| 📽 Description: Important Functions 📽 |
| :--------------------------- |
| In this section we are coding important functions that will be used later in the notebook. |

---


In [ ]:
#defining a function for removing punctuations

punctuations = string.punctuation 

def remove_punctuation(text):
    
    '''removes punctuation and any special characters from a text .
   Args:
        text (str) : containing text data
    
    returns:
        dataframe(df): dataframe with tokens within the specified column.
    '''

    return text.translate(str.maketrans(' ',' ', punctuations))

In [ ]:
#removing numbers
def remove_numbers(df, colname):
    ''' Function removes numbers using regex search
    Arg:
        dataframe (df): dataframe that has a text column
        colname(str): name of the column with text data and emojis
    return
        dataframe (df): dataframe with no numbers from the text data
    '''
   
    num_pattern = r'[0-9]'
    df[colname] = df[colname].replace(to_replace = num_pattern, value = '', regex = True)

    return df

In [ ]:
#removing letters
def remove_letters(df, colname):
    ''' Function removes letters using regex search
    Arg:
        dataframe (df): dataframe that has a text column
        colname(str): name of the column with text data and emojis
    return
        dataframe (df): dataframe with no letters from the text data
    '''
   
    num_pattern = r'^$|[a-zA-Z]|Τσιτσάνης|101次求婚|2006–2007|выбывание|پدر|Начальник|Джа|Девочки|первого'
    df[colname] = df[colname].replace(to_replace = num_pattern, value = np.nan, regex = True)

    return df

In [ ]:
#Counts the number of user ratings
def user_ratings(df, n):
    plt.figure(figsize=(8,6))
    temp_df = df['userId'].value_counts().head(n)
    ax = sns.barplot(x = temp_df.index, y = temp_df, order = temp_df.index, palette = 'bright', edgecolor = "black")
    for i in ax.patches:
        ax.text(i.get_x() + i.get_width()/2., i.get_height(), '%d' % int(i.get_height()), fontsize=11, ha = 'center', va = 'bottom')
    plt.title(f'Top {n} Users by Ratings', fontsize = 14)
    plt.xlabel('User ID')
    plt.ylabel('Number of Ratings')
    print("Number of ratings:\t",df['userId'].value_counts().head(n).sum(),
         "\nTotal number of movies:\t\t", df['movieId'].nunique())
    plt.show()

In [ ]:
def ratings_dist(df, column='rating'):
    plt.figure(figsize=(30,50))
    ax = sns.displot(df[f'{column}'],bins=10, kde = False, color = "red")
    mean = df[f'{column}'].mean()
    median = df[f'{column}'].median()
    plt.axvline(x = mean, label = f'mean {round(mean,2)}' , color = '#1726A0', lw = 3, ls = '--')
    plt.axvline(x = median, label = f'median {median}' , color = '#FFFF00', lw = 3, ls = '--')
    plt.xlim((0.5,5))
    plt.ylim((0,2500000))
    plt.title(f'Distribution of Ratings\n', fontsize = 12)
    plt.xlabel('Rating')
    plt.ylabel('Frequency')
    plt.legend()
    plt.show()

In [ ]:
def plot_ratings(count, n, color = 'red', best = True, method = 'mean'):
    if method == 'mean':
        movie_avg_rates = pd.DataFrame(train.merge(movies, on = 'movieId', how = 'left').groupby(['movieId', 'title'])['rating'].mean())
    else:
        movie_avg_rates = pd.DataFrame(train.merge(movies, on = 'movieId', how = 'left').groupby(['movieId', 'title'])['rating'].median())
    movie_avg_rates['count'] = train.groupby('movieId')['userId'].count().values
    movie_avg_rates.reset_index(inplace = True)
    movie_avg_rates.set_index('movieId', inplace = True)

    # Remove movies that have been rated fewer than n times
    movie_data = movie_avg_rates[movie_avg_rates['count'] > count]
    movie_data.sort_values('rating', inplace = True, ascending = False)
    if best == True:
        plot = movie_data.head(n).sort_values('rating', ascending = True)
        title = 'best rated'
    else:
        plot = movie_data.tail(n).sort_values('rating', ascending = False)
        title = 'worst rated'
    plt.figure(figsize = (10,9))
    sns.scatterplot(x = plot['rating'], y = plot['title'], size = plot['count'], color = color)
    plt.xlabel('rating')
    #plt.ylabel(off)
    plt.tick_params(axis = 'y', which ='both', labelleft = False, labelright = True)
    plt.title(f'Top {n} {title} movies with over {count} ratings', fontsize = 14)
    plt.show()


 <a id="three"></a>
## 3. Loading and Pre-processing Data
<a href=#cont>Back to Table of Contents</a>

---
    
| 📽 Description: Importing Packages & Pre-processing 📽 |
| :--------------------------- |
| In this section we are importing all the csv files with the use of the pandas library that contain our movie's data and converted them into dataframes for our analysis.
  Perform summary statistics on the datasets; checking for null values, descriptive statistics and summary statistics|

---


In [ ]:
#load datasets
train = pd.read_csv('../input/edsa-movie-recommendation-2022/train.csv')
test = pd.read_csv('../input/edsa-movie-recommendation-2022/test.csv')
movies = pd.read_csv('../input/edsa-movie-recommendation-2022/movies.csv')
# specify which columns to include by name
imdb = pd.read_csv('../input/edsa-movie-recommendation-2022/imdb_data.csv', usecols=['movieId','title_cast', 'director', 'runtime'])
tags = pd.read_csv('../input/edsa-movie-recommendation-2022/tags.csv')
print (f'Number of rows in train set: {train.shape[0]}')
print (f'Number of rows in test set: {test.shape[0]}')
print (f'Number of rows in movies test: {movies.shape[0]}')
print (f'Number of rows in imdb set: {imdb.shape[0]}')
print (f'Number of rows in tags set: {tags.shape[0]}')


In [ ]:
test.head()

In [ ]:
train.head()

<div class="alert alert-block alert-success"> 
 <b>why we are merging the datasets:</b>
    
we combined them into one dataset, for the purpose of eda. However since we have multiple csv files which do not contribute to our final model in anyway, we will leave the train and test data as is. The following sections until modeling will only focus on the additional csv files that we will use in our web app
</div>        

In [ ]:
#merge datasets
movies = pd.merge(movies,imdb,on="movieId")
movies = pd.merge(movies,tags,on="movieId")

#view new dataset
movies.head()

In [ ]:
# a quick overview of the data
print("Movies Dataset Info Total Rows | Total Columns | Total Null Values \n")
print(movies.info())
print("---------------------------------------------------------------------")


In [ ]:
movies.shape

<div class="alert alert-block alert-success"> 
 <b>What we have learnt so far:</b> 
    
* The runtime has an average of 100.31 minutes with a minimum of 1 minute and a maximum of 877 minutes.
    
* 75% of the runtime is less than 109 minutes with a median of 98 minutes.

*  large number of feaures lead to overfitting we have to perform feature reduction and/or dimensionality reduction
    
* some features are redundant; they are not necessary for modeling    
    
 </div>      

In [ ]:
#checking null values

print('\nSummary of movies data missing values\n')
print('{}\n'.format(movies.isna().sum()))

print('\nSummary of test data missing values\n')
print('{}\n'.format(test.isna().sum()))

print('\nSummary of train data missing values\n')
print('{}\n'.format(train.isna().sum()))

In [ ]:
missing_values = movies.title_cast.isna().sum()
print(f'Missing Values: {missing_values}  |   Percentage: {round(( missing_values/ movies.title_cast.shape[0]) *100, 2)}%')

In [ ]:
missing_values = movies.director.isna().sum()
print(f'Missing Values: {missing_values}  |   Percentage: {round(( missing_values/ movies.director.shape[0]) *100, 2)}%')

In [ ]:
missing_values = movies.runtime.isna().sum()
print(f'Missing Values: {missing_values}  |   Percentage: {round(( missing_values/ movies.runtime.shape[0]) *100, 2)}%')

In [ ]:
missing_values = movies.tag.isna().sum()
print(f'Missing Values: {missing_values}  |   Percentage: {round(( missing_values/ movies.tag.shape[0]) *100, 2)}%')

In [ ]:
# Remove column name 'timestamp'
movies = movies.drop(['timestamp'], axis=1)

In [ ]:
movies.head()

### Datacleaning

In [ ]:
#creating a new column for year
movies['year'] = movies.title.str.extract('(\(\d\d\d\d\))',expand=False)

In [ ]:
movies['year'] = movies.year.str.extract('(\d\d\d\d)',expand=False)

In [ ]:
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies.head()                                           

In [ ]:
# datacleaning

#removing punctuations from title 
movies.title = movies.title.apply(remove_punctuation)

#removing punctuations from genres
movies.genres = movies.genres.str.replace('|',' ',regex = True)
movies.genres = movies.genres.apply(remove_punctuation)

#removing punctuations from title_cast
movies.title_cast = movies.title_cast.str.replace('|',' ',regex = True)
movies.title_cast = movies.title_cast.astype(str).apply(remove_punctuation)

#removing punctuations from director
movies.director = movies.director.str.replace('|',' ',regex = True)
movies.director = movies.director.astype(str).apply(remove_punctuation)
movies.director = movies.director.dropna()

movies.head()


In [ ]:
missing_values = movies.year.isna().sum()
print(f'Missing Values: {missing_values}  |   Percentage: {round(( missing_values/ movies.year.shape[0]) *100, 2)}%')

<div class="alert alert-block alert-info"> 
 <b>What we have learnt so far:</b> 

    
1. The train data has:
*  <b>*10000038</b>  rows*
*  no missing values
* four columns    
    
2. The test data has:
*  <b>*5000019</b>  rows*
*  two columns   
    
3. The object datatypes are the only columns with empty rows
    
4. We have multiple datasets with multiple features:
*  irrelevant features introduce noise 
*  some features are redundant
*  large number of feaures lead to overfitting
    
**We have to perform feature reduction and/or dimensionality reduction**    
        
</div>         

 <a id="four"></a>
## 4. Exploratory Data Analysis (EDA)
<a href=#cont>Back to Table of Contents</a>

| 📽 *Description: Exploratory Data Analysis* 📽 |
| :--------------------------- |
  In this section, we will perform an in-depth analysis of all the variables in the dataframe. 
  We analyse the data using visual techniques,this is used to discover trends, patterns,
  or to check assumptions with the help of statistical summary and graphical representations.
  Essentially, we want to see how everything is related to each other

<div class="alert alert-block alert-success">
<b>P.S:</b> for in this notebook we will be focusing more on the features related to the movie and how they compare to each other.
</div>


### Ratings
*Check the rating of the movies for any insights*

In [ ]:
movies['runtime'].describe()

In [ ]:
# the funtion for this cell is available in section 2 of the notebook
user_ratings(train,10)

In [ ]:
train['rating'].value_counts(normalize=True)

In [ ]:
train['rating'].unique()

In [ ]:
# the funtion for this cell is available in section 2 of the notebook
ratings_dist(train)

In [ ]:
plot_ratings(10000, 10, 'red', True, 'mean')

In [ ]:
plot_ratings(500, 10,'blue', False, 'mean')

<div class="alert alert-block alert-info"> 
 <b>What we have learnt so far:</b>
    
* The rating scale is between 0.5 and 4.0
* Average rating is 3.5    
* Highest rated movie is Shawshank Redemption
* Lowest rated movie is Battlefield Earth    
</div>    

### Directors
*Check the most popular directors for insights*

In [ ]:
#group director by rating
director = df.groupby(['director'])['rating'].sum().reset_index().sort_values('rating', ascending=False).head(10)
director

In [ ]:
fig = px.bar(director, x = 'rating', y = 'director', title= 'Top Rated Director',color='director', labels={"director": "Director", "rating": "Number of ratings"})
fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.show()

<div class="alert alert-block alert-info"> 
 <b>What we have learnt so far:</b>
    
* Top rated director is Quentin Tarantino
* Steven Spielberg has the least ratings
* Oddly, J.R.R Tolken, the South African author of The Lord of The Rings trilogies as well as The Hobbit books is listed as a director   
</div>

### Movies
*Check the most popular movie yearly for insights*

In [ ]:
# group by title to see which titles are most present in the dataframe
title = df.groupby(['title'])['title'].count().reset_index(name='Count').sort_values('Count', ascending=False).head(10)
title

In [ ]:
fig = px.bar(title, x = 'Count', y = 'title', title= 'Top Movies',color='title', labels={"title": "Titles", "Count": "Number of appearances"})
fig.update_layout(yaxis_categoryorder = 'total ascending')
fig.show()

In [ ]:
import plotly.express as px
fig = px.pie(df, values='title_cast', names='title_cast', title='Title Distribution')
fig.show()

In [ ]:
fig = px.sunburst(df, path=['genres', 'director'], values='rating',
                  color='movieId', hover_data=['title'])
fig.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS
def wordCloud_generator(df, title=None):
    wordcloud = WordCloud(width = 1000, height = 800,
                          background_color ='white',
                          min_font_size = 10
                         ).generate(" ".join(df.values))                      
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=30)
    plt.show()
wordCloud_generator(df['title'], title="Top Titles")

<div class="alert alert-block alert-info"> 
 <b>What we have learnt so far:</b>
    
* Top rated movies are all American
* Top rated movie are what film buffs consider classics
* Top 3 rated movies were released in the early 90s
* Top rated genre or genre combo is Drama    
    
</div>

 <a id="five"></a>
## 5. Model Explanations
<a href=#cont>Back to Table of Contents</a>

| 📽 *Description: Modeling* 📽 |
| :--------------------------- |
  In this section, we are explaining the models that are used in this notebook.
  
  ---

<div class="alert alert-block alert-success">
<b>P.S:</b>
Collaborative Filtering (CF) is a method of making automatic predictions about the interests of a user by learning its preferences (or taste) based on information of his engagements with a set of available items, along with other users’ engagements with the same set of items. 

In other words, CF assumes that, if a person A has the same opinion as person B on some set of issues X={x1,x2,…}, then A is more likely to have B‘s opinion on a new issue y than to have the opinion of any other person that doesn’t agree with A on X


CF algorithms are able to infer users’ hidden preferences and to exploit those preferences to recommend them new potentially-good items, algorithms are best known for their use on e-commerce web sites, where they serve as cornerstones for their recommendation-engines
</div>

### SVD

* Set of supervised learning methods used for classification, regression and outliers detection.
* Effective in high dimensional spaces.
* Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

 <a href='#svd'>click to view code</a>

![](https://www.askpython.com/wp-content/uploads/2020/11/SVD-1.jpg.webp)

### Normal Predicator

 * Prediction algorithm available for recommendation
 * Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal. 
 
 <a href='#np'>click to view code</a>

### Baseline Only

* Algorithm predicting the baseline estimate for given user and item.
* Common baseline models include linear regression when predicting continuous values, logistic regression when classifying structured data, pretrained convolutional neural networks for vision related tasks, and recurrent neural networks and gradient boosted trees for sequence modeling

<a href='#BO'>click to view code</a>

### Non-negative Matrix Factorization

* The objective of NMF is dimensionality reduction and feature extraction
* Intuitively, NMF assumes that the original input is made of a set of hidden features, represented by each column of W matrix and each column in H matrix represents the ‘coordinates of a data point’ in the matrix W.
* Is used in major applications such as image processing, text mining, spectral data analysis and many more

<a href='#nmf'>click to view code</a>


### Co-Clustering (or Biclustering) 

* Simultaneous clustering of the rows and columns of a matrix. 
* Can be seen as a method of co-grouping two types of entities simultaneously, based on similarity of their pairwise interactions.
* Extremely useful when the above mentioned pairwise interactions signal is sparse

<a href='#co_clustering'>click to view code</a>

### Slope One

* A simple yet accurate collaborative filtering algorithm
* Slope One algorithm uses simple linear regression model to solve data sparisity problem.
* Used mostly in collaborative filtering

<a href='#slope_one'>click to view code</a>



 <a id="six"></a>
## 6. Modeling
<a href=#cont>Back to Table of Contents</a>

| 📽 *Description: Model Performance* 📽 |
| :--------------------------- |
  In this section, we peform the modeling with the train and test sets, then pick the best performing model.
  
  ---

![](https://www.bbvaapimarket.com/wp-content/uploads/2016/04/cibbva_modelo.png)

In [ ]:
# Load the train dataset
from surprise import Reader, Dataset
reader = Reader()
model = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [ ]:
# sample random trainset and testset
train_set,test_set = train_test_split(model,test_size=0.25,random_state=42)

### Singular Value Decomposition (SVD)
<a id='svd'></a>

In [ ]:
# svd
svd = SVD(n_epochs = 30, n_factors = 200, init_std_dev = 0.05, random_state=42)
svd.fit(trainset)
pred = svd.test(testset)
svd_rmse = accuracy.rmse(pred)

### Normal Predictor
<a id='np'></a>

In [ ]:
# np
np = NormalPredictor()
np.fit(trainset)
pred = np.test(testset)
np_rmse = accuracy.rmse(pred)

### Baseline Only
<a id='BO'></a>

In [ ]:
# baseline
bsl = {'method': 'sgd','n_epochs': 40}
BO = BaselineOnly()
BO.fit(trainset)
pred = BO.test(testset)
BO_rmse = accuracy.rmse(pred)

 ### Non-negative Matrix Factorization
 
 <a id='nmf'></a>

In [ ]:
# nmf
nmf = NMF()
nmf.fit(trainset)
pred = nmf.test(testset)
nmf_rmse = accuracy.rmse(pred)

### Slope One

<a id='slope_one'></a>

In [ ]:
# slope one
slope = SlopeOne()
slope.fit(trainset)
pred = slope.test(testset)
slope_rmse = accuracy.rmse(pred)

### Co-Clustering
<a id='co_clustering'></a>

In [ ]:
# co-clustering
c_cluster = CoClustering(random_state=42)
c_cluster.fit(trainset)
pred = c_cluster.test(testset)
cluster_rmse = accuracy.rmse(pred)

 <a id="seven"></a>
## 7. Model Performance / Validation
<a href=#cont>Back to Table of Contents</a>

| 📽 *Description: Model Performance* 📽 |
| :--------------------------- |
  In this section, we analyse the peformance of all the models we have coded.
  
  ---



In [ ]:
# Compare RMSE values between models
fig,axis = plt.subplots(figsize=(8, 5))
clusters = ['SVD','NormalPredictor','BaselineOnly','NMF','SlopeOne','CoClustering']
rmse_y = [svd_rmse, np_rmse, BO_rmse, nmf_rmse, slope_rmse, cluster_rmse]
ax = sns.barplot(x = clusters, y = rmse_y, palette = 'bright', edgecolor = 'black')
plt.title('Model Performance', fontsize = 12)
plt.xticks(rotation=90)
plt.ylabel('RMSE')
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2, p.get_y() + p.get_height(), round(p.get_height(),2), fontsize=12, ha="center", va='bottom')
    
plt.show()

<div class="alert alert-block alert-info"> 
 <b>What we have learnt:</b>
    
  *  In statistics, lower values of RMSE indicate better fit. RMSE is a good measure of how accurately the model predicts the response.
  * This metric that tells us the average distance between the predicted values from the model and the actual values in the dataset.
  *  The lower the RMSE, the better a given model is able to “fit” a dataset.
  
    *From our models, **SVD** performs the best!*

</div>

### Cross Validation

In [ ]:
svd_val = SVD(n_epochs = 40, n_factors = 200, init_std_dev = 0.05, random_state = 42)
CV_ = cross_validate(svd_val, data, measures = ['RMSE'], cv = 5, verbose = True)

In [ ]:
param_grid = {'n_epochs':[40],
              'n_factors':[400], 
              'init_std_dev':[0.005], 
              'random_state':[42]} 
grid_SVD = GridSearchCV(SVD, cv = 5, measures = ['rmse'], param_grid = param_grid, n_jobs = -1)
grid_SVD.fit(data)
print('Best score:\t', grid_SVD.best_score['rmse'])
print('Best parameters:\t', grid_SVD.best_params['rmse'])

In [ ]:
svd_test = SVD(n_epochs = 40, n_factors = 400, init_std_dev = 0.005, random_state=42)
svd_test.fit(trainset)
predictions = svd_test.test(testset)
rmse = accuracy.rmse(predictions)

<div class="alert alert-block alert-success">
<b>Cross-validation:</b> By using Cross-Validation, we are able to get more metrics and draw important conclusion both about our algorithm and our data.
</div>



 <a id="eight"></a>
## 8. References
<a href=#cont>Back to Table of Contents</a>

| 📽 *Description: References* 📽 |
| :--------------------------- |
  In this section, we have listed all the refrences used throught this notebook for futher reading.
  
  ---
  
    
1. [What Are Recommender System](https://www.analyticssteps.com/blogs/what-are-recommendation-systems-machine-learning)
2. [Explained - Working and Advantages of a Recommendation Engine](https://medium.com/geekculture/explained-working-and-advantages-of-a-recommendation-engine-16cbff7796c)
3. [(5) Reasons why you should use Cross-Validation](https://towardsdatascience.com/5-reasons-why-you-should-use-cross-validation-in-your-data-science-project-8163311a1e79)
4. [SVD](https://scikit-learn.org/stable/modules/svm.html)
5. [Collaborative Filtering](https://datasciencemadesimpler.wordpress.com/tag/co-clustering/)

                                                                   *FIN*  